# ACCESS-MOPPeR Getting Started

## Set up configuration

When you first import `access_mopper` in a Python environment, the package will automatically create a `user.yml` file in your home directory (`~/.mopper/user.yml`).  

During this initial setup, you will be prompted to provide some basic information, including:  
- Your name  
- Your email address  
- Your work organization
- Your ORCID

This information is stored in `user.yml` and will be used as global attributes in the files generated during the CMORisation process. This ensures that each CMORised file includes metadata identifying who performed the CMORisation, allowing us to track data provenance and follow up with the responsible person if needed.

In [2]:

import access_mopper as mop

## Create an Experiment

In the context of CMORising data with **ACCESS-MOPPeR**, the next step involves creating an **experiment**. An **experiment** is a **dataclass** that defines the type of CMIP experiment you're working with. It plays a critical role in ensuring that the necessary metadata and configurations are applied when processing model output for CMORisation.

### 1. **What is an Experiment?**
An experiment in **ACCESS-MOPPeR** contains key information about the CMIP experiment you're trying to CMORise. This includes critical data such as:
- **experiment_id**: For example, `"piControl-spinup"` in the context of the **ESM1.6** spin-up run.
- **Additional Metadata**: The experiment dataclass also includes other metadata that will be stored as **global attributes** in the final CMORised output files. This information helps track the experiment, ensuring that relevant details (e.g., model version, configuration, initialization) are properly recorded in the processed data files.

### 2. **Creating an ESM1.6 Experiment**
For an **ESM1.6** experiment, you would define the experiment using an instance of the `ACCESS-ESM16-CMIP6` class. This class is designed specifically to handle the configurations associated with **ACCESS-ESM1.6**, and it pre-fills important details that will be attached to the output files after CMORisation. 

By using this class, you'll avoid manually entering configuration information, as it automates this process. These details include, but are not limited to:
- The experiment's **id** (e.g., `"piControl-spinup"`).
- The relevant **model configuration**, **forcing**, and **initialization** settings that are necessary to ensure proper CMORisation.

### 3. Example:

In [3]:
file_pattern = "../tests/data/esm1-6/atmosphere/aiihca.pa-101909_mon.nc"

In [3]:
from access_mopper.configurations import ACCESS_ESM16_CMIP6

experiment = ACCESS_ESM16_CMIP6(experiment_id = "piControl-spinup", 
                                realization_index="1", 
                                initialization_index="1", 
                                physics_index="1", 
                                forcing_index="1", 
                                parent_mip_era="no parent",
                                parent_activity_id="no parent",
                                parent_experiment_id= "no parent",
                                parent_source_id = "no parent",
                                parent_variant_label = "no parent",
                                parent_time_units = "no parent",
                                branch_method = "no parent",
                          )

### 3. **Serializing the Experiment**
Once the experiment is defined using the `ACCESS-ESM16-CMIP6` class, it is serialized into a **JSON** file. This serialized JSON file contains all the metadata and configurations associated with the experiment. The JSON file will then be used by the **CMORiser** to generate the CMORised output data files.

This serialized file ensures that the necessary context, experiment configurations, and metadata are applied consistently throughout the CMORisation process.

This experiment setup ensures that the required information (such as experiment configurations and global attributes) is automatically integrated into the CMORisation process, making the generated files compliant with the CMIP standards.

In summary, the **experiment** dataclass defines the core properties of the CMIP experiment you're CMORising, with the **ACCESS-ESM16-CMIP6** class pre-filling essential details. The resulting experiment is serialized into a JSON file, ensuring that these attributes are consistently applied during CMORisation.

In [4]:
experiment.save_to_file("experiment.json")

Data saved to experiment.json


## Explore Mapping information for a specific variable

In [5]:
experiment.mapping_info("Amon.hur")

### Mapping Information for `Amon.hur`
    - **Compound Name**: `Amon.hur`
    - **CF Standard Name**: `N/A`
    - **Required Variables**: `fld_s30i206`
    - **Formula**: `fld_s30i206`

## CMORising files for a Land variable (baresoilFrac)

To **CMORise** the content of outputs for a specific variable, such as `baresoilFrac` from the **Lmon** (monthly mean) dataset, we follow a structured approach using **CMIP** standards and leveraging the **compound name** for the variable. Let's break this down step by step:

### 1. **Why Use a Compound Name?**
In CMIP, each variable is not just defined by a name (e.g., `baresoilFrac`) but by a **compound name** that includes both the variable and its associated dimensions, ensuring that we look at the specific requirements set by CMIP for that variable. 

A **compound name** like `"Lmon.baresoilFrac"` helps:
- **Define the frequency and variable**: `Lmon` indicates the dataset is for **monthly** means (monthly time step).
- **Address the specific variable**: `baresoilFrac` is the specific variable related to bare soil fraction.

This compound naming ensures that the CMORiser will apply the correct metadata and compliance rules for the given variable, ensuring it matches the CMIP specifications.

### 2. **CMORisation Example**

To CMORise the output data for the `baresoilFrac` variable from the **Lmon** dataset, we'll use the `cmorise` function. Below is an example of how you would use this function in your script:

In [6]:
import glob
#from access_mopper.configurations import cmorise
val = experiment.cmorise(file_paths=glob.glob(file_pattern),
                         compound_name= "Lmon.baresoilFrac", 
                         cmor_dataset_json= "experiment.json",
                         mip_table="CMIP6_Lmon.json")

Stored in: MOPPeR_outputs/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl-spinup/r1i1p1f1/Lmon/baresoilFrac/gn/v20250423/baresoilFrac_Lmon_ACCESS-ESM1-5_piControl-spinup_r1i1p1f1_gn_101909-101909.nc



! ------
! All files were closed successfully. 
! ------
! 



### 3. **How It Works:**
When you run this code:
- The `cmorise` function will first use the **file paths** obtained from the `glob` pattern to locate the necessary data files (those containing the `baresoilFrac` variable).
- The **compound name** (`Lmon.baresoilFrac`) will ensure that the CMORiser applies the correct rules and metadata for the **baresoilFrac** variable.
- The experiment details and global attributes are read from the **`experiment.json`** file, ensuring that the resulting CMORised output files are tagged with the appropriate metadata (such as the experiment id, model configuration, and other details).
- The **MIP table** (`CMIP6_Lmon.json`) will guide the CMORisation process, ensuring that the output files adhere to CMIP6 standards for Lmon-type variables.

## CMORising files for an Atmosphere variable (hur)


In [7]:
file_pattern = "../tests/data/esm1-6/atmosphere/aiihca.pa-101909_mon.nc"

In [8]:
val = experiment.cmorise(file_paths=glob.glob(file_pattern),
                         compound_name= "Amon.hur", 
                         cmor_dataset_json= "experiment.json",
                         mip_table="CMIP6_Amon.json")

Stored in: MOPPeR_outputs/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl-spinup/r1i1p1f1/Amon/hur/gn/v20250423/hur_Amon_ACCESS-ESM1-5_piControl-spinup_r1i1p1f1_gn_101909-101909.nc


/home/romain/PROJECTS/ACCESS-MOPPeR/mopper_conda/lib/python3.11/site-packages/cmor/pywrapper.py:759: UserWarning: Error: your data shape ((1, 19, 144, 192)) does not match the expected variable shape ([144, 192, 0, 19])
Check your variable dimensions before caling cmor_write
  warnings.warn(msg)

! ------
! All files were closed successfully. 
! ------
! 


## CMORising files for an Ocean variable (zos) (ACCESS-OM2 In development)

In [11]:
file_pattern = "/home/romain/PROJECTS/ACCESS-MOPPeR/Test_data/cj877/history/ocn/ocean-2d-sea_level-1-monthly-mean-ym_0326_01.nc"

In [12]:
experiment.supergrid = "/home/romain/PROJECTS/ACCESS-MOPPeR/grids/access-om2/input_20201102/mom_025deg/ocean_hgrid.nc"

In [13]:
val = experiment.cmorise_ocean(file_paths=glob.glob(file_pattern),
                               compound_name= "Omon.zos", 
                               cmor_dataset_json= "experiment.json",
                               mip_table="CMIP6_Omon.json")

Stored in: MOPPeR_outputs/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl-spinup/r1i1p1f1/Omon/zos/gn/v20250423/zos_Omon_ACCESS-ESM1-5_piControl-spinup_r1i1p1f1_gn_032601-032606.nc



! ------
! All files were closed successfully. 
! ------
! 


## CMORising files for an Ocean variable (tos) (ACCESS-OM3 In development)

In [4]:
from access_mopper.configurations import ACCESS_OM3_CMIP6

experiment = ACCESS_OM3_CMIP6(experiment_id = "historical", 
                                realization_index="1", 
                                initialization_index="1", 
                                physics_index="1", 
                                forcing_index="1", 
                                parent_mip_era="CMIP6",
                                parent_activity_id="CMIP",
                                parent_experiment_id= "piControl",
                                parent_source_id = "ACCESS-ESM1-5",
                                parent_variant_label = "r1i1p1f1",
                                parent_time_units = "days since 0101-1-1",
                                branch_method = "standard",
                                branch_time_in_child = 0.0,
                                branch_time_in_parent = 0.0)

In [5]:
experiment.save_to_file("experiment.json")

Data saved to experiment.json


In [6]:
experiment.mapping_info("Omon.tos")

### Mapping Information for `Omon.tos`
    - **Compound Name**: `Omon.tos`
    - **CF Standard Name**: `N/A`
    - **Required Variables**: `tos`
    - **Formula**: `tos`

In [7]:
file_pattern = "/g/data/tm70/ml0072/archive/access-om3/025deg_jra55do_ryf_2024.09.1_build_w_Ezhil_first_bathy_edit/output014/access-om3.mom6.2d.tos.1mon.mean.*.nc"

In [8]:
experiment.supergrid = "/g/data/vk83/prerelease/configurations/inputs/access-om3/mom/grids/mosaic/global.25km/2025.01.30/ocean_hgrid.nc"

In [9]:
import glob
val = experiment.cmorise(file_paths=file_pattern,
            compound_name= "Omon.tos", 
            cmor_dataset_json= "experiment.json",
            mip_table="CMIP6_Omon.json")


C Traceback:
In function: _CV_checkSourceID
! called from: cmor_write
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: Your input attribute "source" with value 
! "ACCESS-OM3 (2025): (GFDL-MOM5, tripolar primarily 0.25deg; 1440 x 1142 longitude/latitude; 75 levels; top grid cell 0-10 m), ocnBgchem: none, seaIce: CICE5.1.2 (same grid as ocean)" will be replaced with value 
! "ACCESS-CM2 (2019): 
aerosol: UKCA-GLOMAP-mode
atmos: MetUM-HadGEM3-GA7.1 (N96; 192 x 144 longitude/latitude; 85 levels; top level 85 km)
atmosChem: none
land: CABLE2.5
landIce: none
ocean: ACCESS-OM2 (GFDL-MOM5, tripolar primarily 1deg; 360 x 300 longitude/latitude; 50 levels; top grid cell 0-10 m)
ocnBgchem: none
seaIce: CICE5.1.2 (same grid as ocean)".
! 
! 
!  See Control Vocabulary JSON file.(/g/data/xp65/public/apps/med_conda/envs/analysis3-25.05/lib/python3.11/site-packages/access_mopper/cmor_tables/CMIP6_CV.json)
! 
!
!!!!!!!!!!!!!!!!!!!!!!!!!



Stored in: MOPPeR_outputs/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Omon/tos/gn/v20250502/tos_Omon_ACCESS-CM2_historical_r1i1p1f1_gn_192801-192912.nc


! ------
! CMOR is now closed.
! ------
! During execution we encountered:
!   1 Warning(s)
!   0 Error(s)
! ------
! Please review them.
! ------
! 
